# Ensemble Models Training

This notebook trains two ensemble models:
1. **Stacking Ensemble**: Combines all base models using meta-learner (5-fold CV)
2. **Weighted Averaging Ensemble**: Optimizes linear combination weights

## Base Models:
- Logistic Regression
- Random Forest
- XGBoost
- Neural Network (Champion)

---

## Section 1: Stacking Ensemble

**Method**: 5-fold cross-validation with Logistic Regression meta-learner

**How it works**:
1. Generate out-of-fold predictions from base models
2. Train meta-learner on these predictions
3. Combine base model outputs for final prediction

**Expected runtime**: ~5-10 minutes

In [ ]:
import sys
from pathlib import Path

# Add project root to path
project_root = Path.cwd().parent.parent
sys.path.insert(0, str(project_root / 'credit_risk_fyp'))

# Import and run stacking ensemble training
from src.train_stacking_ensemble import main as train_stacking

print("Starting Stacking Ensemble training...\n")
train_stacking()
print("\n✓ Stacking Ensemble training complete!")

INFO:data_pipeline:Loading processed data...


Starting Stacking Ensemble training...


STACKING ENSEMBLE MODEL TRAINING

1. Loading processed data...


INFO:data_pipeline:✓ Training data (SMOTE-balanced): (364313, 103)
INFO:data_pipeline:✓ Validation data: (75000, 103)
INFO:data_pipeline:✓ Test data: (100000, 103)


Training set: (364313, 103)
Validation set: (75000, 103)
Test set: (100000, 103)

2. Loading base models...

LOADING BASE MODELS
✓ Loaded Logistic Regression
✓ Loaded Random Forest
✓ Loaded XGBoost
⚠ TensorFlow not available in this environment, skipping Neural Network model
  (Ensemble will work with remaining models)

Total models loaded: 3

3. Generating meta-features...

GENERATING META-FEATURES WITH 5-FOLD CROSS-VALIDATION

Base models: logistic_regression, random_forest, xgboost
Folds: 5
Training samples: 364,313

[1/3] Processing logistic_regression...
  Fold 1/5... Done
  Fold 2/5... Done
  Fold 3/5... Done
  Fold 4/5... Done
  Fold 5/5... Done
  Generating predictions for validation and test sets...
  ✓ logistic_regression meta-features complete

[2/3] Processing random_forest...
  Fold 1/5... Done
  Fold 2/5... Done
  Fold 3/5... Done
  Fold 4/5... Done
  Fold 5/5... Done
  Generating predictions for validation and test sets...


[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    1.0s
[Parallel(n_jobs=8)]: Done 200 out of 200 | elapsed:    1.1s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    1.3s
[Parallel(n_jobs=8)]: Done 200 out of 200 | elapsed:    1.4s finished


  ✓ random_forest meta-features complete

[3/3] Processing xgboost...
  Fold 1/5... Done
  Fold 2/5... Done
  Fold 3/5... Done
  Fold 4/5... Done
  Fold 5/5... Done
  Generating predictions for validation and test sets...
  ✓ xgboost meta-features complete

META-FEATURES GENERATION COMPLETE
Train meta-features shape: (364313, 3)
Validation meta-features shape: (75000, 3)
Test meta-features shape: (100000, 3)

4. Training meta-learner...

TRAINING META-LEARNER (LOGISTIC REGRESSION)
✓ Meta-learner training complete

Meta-learner coefficients:
  Logistic Regression: -1.2427
  Random Forest: 3.3186
  XGBoost: 4.4334

5. Making predictions...

6. Finding optimal threshold on validation set...

Finding optimal threshold (optimizing tpr_fpr)...
  Progress: 0/100
  Progress: 20/100
  Progress: 40/100
  Progress: 60/100
  Progress: 80/100
Optimal threshold: 0.2363

VALIDATION SET RESULTS
AUC-ROC:          0.7247
Precision:        0.3181
Recall (TPR):     0.6950
F1-Score:         0.4365
FPR:    

---

## Section 2: Weighted Averaging Ensemble

**Method**: Optimize linear combination weights using scipy

**How it works**:
1. Get predictions from all base models
2. Optimize weights to maximize AUC-ROC on validation set
3. Apply weighted average: `final_pred = w1*LR + w2*RF + w3*XGB + w4*NN`

**Expected runtime**: ~2-3 minutes

In [2]:
# Import and run weighted ensemble training
from src.train_weighted_ensemble import main as train_weighted

print("Starting Weighted Averaging Ensemble training...\n")
train_weighted()
print("\n✓ Weighted Averaging Ensemble training complete!")

INFO:data_pipeline:Loading processed data...


Starting Weighted Averaging Ensemble training...


WEIGHTED AVERAGING ENSEMBLE MODEL TRAINING

1. Loading processed data...


INFO:data_pipeline:✓ Training data (SMOTE-balanced): (364313, 103)
INFO:data_pipeline:✓ Validation data: (75000, 103)
INFO:data_pipeline:✓ Test data: (100000, 103)


Training set: (364313, 103)
Validation set: (75000, 103)
Test set: (100000, 103)

2. Loading base models...

LOADING BASE MODELS
✓ Loaded Logistic Regression
✓ Loaded Random Forest
✓ Loaded XGBoost
⚠ TensorFlow not available in this environment, skipping Neural Network model
  (Ensemble will work with remaining models)

Total models loaded: 3

3. Getting base model predictions...

GETTING BASE MODEL PREDICTIONS
Base models: logistic_regression, random_forest, xgboost

[1/3] logistic_regression...
  ✓ Predictions shape: val=(75000,), test=(100000,)

[2/3] random_forest...


[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    1.2s
[Parallel(n_jobs=8)]: Done 200 out of 200 | elapsed:    1.3s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    1.5s
[Parallel(n_jobs=8)]: Done 200 out of 200 | elapsed:    1.6s finished


  ✓ Predictions shape: val=(75000,), test=(100000,)

[3/3] xgboost...
  ✓ Predictions shape: val=(75000,), test=(100000,)

BASE PREDICTIONS COMPLETE

4. Optimizing ensemble weights...

OPTIMIZING WEIGHTS (metric: AUC_ROC)
Initial weights (equal): [0.33333333 0.33333333 0.33333333]
Running optimization...

OPTIMIZATION RESULTS

Optimized weights:
  Logistic Regression: 0.0000 (0.0%)
  Random Forest: 0.4483 (44.8%)
  XGBoost: 0.5517 (55.2%)

Sum of weights: 1.000000
Optimization score: 0.7247

5. Making weighted predictions...

6. Finding optimal threshold on validation set...

Finding optimal threshold (optimizing tpr_fpr)...
  Progress: 0/100
  Progress: 20/100
  Progress: 40/100
  Progress: 60/100
  Progress: 80/100
Optimal threshold: 0.3696

VALIDATION SET RESULTS
AUC-ROC:          0.7247
Precision:        0.3226
Recall (TPR):     0.6788
F1-Score:         0.4374
FPR:              0.3548
Specificity:      0.6452

7. Evaluating on test set...

TEST SET RESULTS
AUC-ROC:          0.7227


---

## Training Complete!

Both ensemble models have been trained and results saved to:
- `credit_risk_fyp/results/stacking_ensemble_metrics.pkl`
- `credit_risk_fyp/results/weighted_ensemble_metrics.pkl`

**Next Step**: Run `model_comparison.ipynb` to compare all 6 models:
1. Logistic Regression
2. Random Forest
3. XGBoost
4. Neural Network
5. Stacking Ensemble
6. Weighted Averaging Ensemble